In [ ]:
%pylab inline --no-import-all

from ctypes import c_float

# import numpy as np
from astropy.io import fits


import ROOT
from ROOT import gSystem, TFile, TGraphAsymmErrors


from root_numpy import hist2array


In [ ]:
if gSystem.Load("libVEGASCommon.dylib"):
    print("Problem loading VEGAS Common libraries - please check this before proceeding")
if gSystem.Load("libVEGASStage6.dylib"):
    print("Problem loading VEGAS Stage 6 libraries - please check this before proceeding")
if gSystem.Load("libVEGASStage5.dylib"):
    print("Problem loading VEGAS Stage 5 libraries - please check this before proceeding")
from ROOT import VARootIO, VAEffectiveAreaManager, VAEASimpleParameterData



# Compatability
Works with VEGAS v2.5.7 or later.

# Data format
the data format is defined at : https://gamma-astro-data-formats.readthedocs.io/en/latest/

# Task List

## Top priorities
* Time Cuts from ROOT file
* Average Azimuth assumes southerly source ....
* Convert to format so can read through a stage 6 runlist file
* Check validity of 30 min runs for EA (compare decent Crab runlist in 30 vs 10 min runs @ different zenith angles).
* Adding ED capability.

## Validation
* Check high stats crab spectra
* Check other spectra
* Check significances/sky maps
* Upper limits

## Documentation
* Save a number of notebooks that show how to do conversion and analysis

## Wish list
* All offset - this will require reworking events list for saving noises etc.
* Event Types - how do spectra compare when produced as "all events" vs. breaking up into 2, 3 and 4 tel events.
* Get window size for noise from root/ea file - at the moment assume it is 7

In [ ]:
windowSizeForNoise = 7

def decodeConfigMask(mask=15):
    '''Decode the telescope config mask to find the telescpes in the array'''
    tels = []
    if mask >= 8:
        tels.append(4)
        mask -= 8
    if mask >= 4:
        tels.append(3)
        mask -= 4
    if mask >= 2:
        tels.append(2)
        mask -= 2
    if mask >= 1:
        tels.append(1)
        mask -= 1
    return sorted(tels)


def produceTelList(mask):
    '''Convert the list of telescopes into a string for FITS header'''
    telList = ""
    for tel in decodeConfigMask(mask):
        telList += "T" + str(tel) + ","
    return telList[:-1]

# File Generation

## First we need to generate the primary HDU

This contains the information about who wrote the file and the standards that it is written too.
The basic header has some information, we need to complete it to have the following

<pre>
SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
TELESCOP= 'VERITAS'            / Telescope                                      
LICENSE = '        '           / Copyright (c) 2018,The VERITAS Collaboration     
</pre>

In [ ]:
i = 3

runs = [54809, 57993, 58456, 59523]
atms = [21, 22, 21, 21]

run = str(runs[i])
atm = str(atms[i])
st5File = "VEGAS/"+run+".med.ED.050.St5_Stereo.root"
eaFile = "VEGAS/EA_na"+atm+"stan_medPoint_050_ED_GRISU.root"
outfile = 'VEGAS/DL3/'+run+'_DL3.fits'

In [ ]:
hdu0 = fits.PrimaryHDU()
hdu0.header.set('TELESCOP', 'VERITAS', 'Telescope')
hdu0.header.set('LICENSE ', '', 'Copyright (c) 2018,The VERITAS Collaboration')
hdu0.header['COMMENT'] = "FITS (Flexible Image Transport System) format is defined in 'Astronomy"
hdu0.header['COMMENT'] = "and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H"

In [ ]:
hdu0.header

## Event Table 
The second hdu is the event table - this includes all of the events that pass the gamma/hadron selection cuts.

To load this we need to read in the VEGAS stage 5 file and select the keys of interest.  Then this can be saved into a table and then the hdu.  

We also need to put a lot of information into the header about the observations.

In [ ]:
vegasFileIO = VARootIO(st5File, True)
runHeader = vegasFileIO.loadTheRunHeader()
selectedEventsTree = vegasFileIO.loadTheCutEventTree()
qStatsData = vegasFileIO.loadTheQStatsData()

arrayInfo = vegasFileIO.loadTheArrayInfo(0)
pixelData = vegasFileIO.loadThePixelStatusData()

In [ ]:
vegasFileIO.loadTheRunHeader

In [ ]:
runHeader.printRunHeader()

In [ ]:
selectedEventsTree.Print()

### First we need to generate an array of the data

In [ ]:
evNumArr = []
timeArr = []
raArr = []
decArr = []
azArr = []
altArr = []
energyArr = []
detXArr = []
detYArr = []
nTelArr = []

### Now we can populate the array from the ROOT file

Note: We also need to save some useful information for the average altitude, azimuth, RA and Dec.

I am not sure how best to do the average of the azimuth - I have ignored this for now ...

In [ ]:
avAlt = []
avAz = []
avRA = []
avDec = []
for ev in selectedEventsTree:
    evNumArr.append(ev.S.fArrayEventNum)
    timeArr.append(float(ev.S.fTime.getDayNS())/1e9)
    raArr.append(np.rad2deg(ev.S.fDirectionRA_J2000_Rad))
    decArr.append(np.rad2deg(ev.S.fDirectionDec_J2000_Rad))
    azArr.append(np.rad2deg(ev.S.fDirectionAzimuth_Rad))
    altArr.append(np.rad2deg(ev.S.fDirectionElevation_Rad))
    energyArr.append(ev.S.fEnergy_GeV / 1000.)
    detXArr.append(ev.S.fDirectionXCamPlane_Deg)
    detYArr.append(ev.S.fDirectionYCamPlane_Deg)
    nTelArr.append(ev.S.fImages)
    
    avAlt.append(ev.S.fArrayTrackingElevation_Deg)
    avAz.append(ev.S.fArrayTrackingAzimuth_Deg)
    avRA.append(ev.S.fArrayTrackingRA_J2000_Rad)
    avDec.append(ev.S.fArrayTrackingDec_J2000_Rad)
        
avAlt = np.mean(avAlt)
# Calculate average azimuth angle from average vector on a circle
# https://en.wikipedia.org/wiki/Mean_of_circular_quantities
avAz = np.rad2deg(np.arctan2(np.sum(np.sin(avAz)),np.sum(np.cos(avAz))))
avAz = avAz if avAz > 0 else avAz + 360

avRA = np.rad2deg(np.mean(avRA))
avDec = np.rad2deg(np.mean(avDec))

### Now we need to generate an HDU

In [ ]:
hdu1 = fits.BinTableHDU.from_columns([
    fits.Column(name='EVENT_ID', format='1J', array=evNumArr), 
    fits.Column(name='TIME', format='1D', array=timeArr, unit="s"), 
    fits.Column(name='RA', format='1E', array=raArr, unit = "deg"), 
    fits.Column(name='DEC', format='1E', array=decArr, unit = "deg"), 
    fits.Column(name='ALT', format='1E', array=altArr, unit = "deg"), 
    fits.Column(name='AZ', format='1E', array=azArr, unit = "deg"), 
    fits.Column(name='ENERGY', format='1E', array=energyArr, unit = "TeV"), 
    fits.Column(name='DETX', format='1E', array=detXArr, unit = "deg"), 
    fits.Column(name='DETY', format='1E', array=detYArr, unit = "deg"),
    fits.Column(name="EVENT_TYPE", format="1J", array=nTelArr)
])
hdu1.name = "EVENTS"

### Header Information

In [ ]:
hdu1.header.set('OBS_ID  ', runHeader.getRunNumber(), 'Run Number')
hdu1.header.set('TELESCOP', 'VERITAS', 'Data from VERITAS')

startTime = runHeader.getStartTime()
endTime = runHeader.getEndTime()

startTime_s = float(startTime.getDayNS()) / 1e9
endTime_s = float(endTime.getDayNS()) / 1e9

print(startTime_s,endTime_s)
startTime_s = float(startTime.getDayNS()) / 1e9
endTime_s = float(endTime.getDayNS()) / 1e9
hdu1.header.set('DATE-OBS',
                startTime.getString().split()[0],
                'start date (UTC) of obs yy-mm-dd')
hdu1.header.set('TIME-OBS',
                startTime.getString().split()[1],
                'start time (UTC) of obs hh-mm-ss')
hdu1.header.set('DATE-END',
                endTime.getString().split()[0],
                'end date (UTC) of obs yy-mm-dd')
hdu1.header.set('TIME-END',
                endTime.getString().split()[1],
                'end time (UTC) of obs hh-mm-ss')

hdu1.header.set('TSTART  ',
                startTime_s,
                'mission time of start of obs [s]')
hdu1.header.set('TSTOP   ',
                endTime_s,
                'mission time of end of obs [s]')
hdu1.header.set('MJDREFI ',
                int(startTime.getMJDInt()), 'int part of reference MJD [days]')
hdu1.header.set('MJDREFF ', 0., 'fractional part of reference MJD [days]')

hdu1.header.set('TIMEUNIT', 's', 'time unit is seconds since MET start')
hdu1.header.set('TIMESYS ', 'utc', 'time scale is UTC')
hdu1.header.set('TIMEREF ', 'local', 'local time reference')

hdu1.header.set('ONTIME  ', 
                endTime_s - startTime_s,
                'time on target (including deadtime)')
hdu1.header.set('LIVETIME', runHeader.pfRunDetails.fRunNominalLiveTimeSeconds,
                '(dead=ONTIME-LIVETIME) [s] ')
hdu1.header.set('DEADC   ', runHeader.getLiveTimeFrac(),
                'average deadtime fraction [] ')

hdu1.header.set('OBJECT  ', runHeader.getSourceId(), 'observed object')

hdu1.header.set('RA_PNT  ', avRA, 'observation position RA [deg]')
hdu1.header.set('DEC_PNT ', avDec, 'observation position DEC [deg]')
hdu1.header.set('ALT_PNT ', avAlt, 'average altitude of pointing [deg]')
hdu1.header.set('AZ_PNT  ', avAz, 'average azimuth of pointing [deg]')

hdu1.header.set('RA_OBJ  ',
                np.rad2deg(runHeader.getSourceRA()),
                'observation position RA [deg]')
hdu1.header.set('DEC_OBJ ',
                np.rad2deg(runHeader.getSourceDec()),
                'observation position DEC [deg]')

# get the list of telescopes that participate in the event
hdu1.header.set('TELLIST',
                produceTelList(runHeader.fRunInfo.fConfigMask),
                'comma-separated list of tel IDs')
hdu1.header.set('N_TELS', runHeader.pfRunDetails.fTels,
                'number of telescopes in event list')

# other info - weather? pointing mode

hdu1.header.set('EUNIT   ', 'TeV', 'energy unit')
hdu1.header.set('GEOLAT  ', np.rad2deg(arrayInfo.longitudeRad()), 'longitude of array center [deg]')
hdu1.header.set('GEOLON  ', np.rad2deg(arrayInfo.latitudeRad()), 'latitude of array center [deg]')
hdu1.header.set('ALTITUDE', arrayInfo.elevationM(), 'altitude of array center [m]')

# What are these for? - May note be needed, leave out for now.
# hdu1.header.set('DSTYP1', 'TIME    ', 'Data selection type')
# hdu1.header.set('DSUNI1', 's       ', 'Data selection unit')
# hdu1.header.set('DSVAL1', 'TABLE   ', 'Data selection value')
# hdu1.header.set('DSREF1', ':GTI    ', 'Data selection reference')
# hdu1.header.set('DSTYP2', 'POS(RA,DEC)', 'Data selection type')
# hdu1.header.set('DSUNI2', 'deg     ', 'Data selection unit')
# hdu1.header.set('DSVAL2', 'CIRCLE(83.63,22.01,5)', 'Data selection value')
# hdu1.header.set('DSTYP3', 'ENERGY  ', 'Data selection type')
# hdu1.header.set('DSUNI3', 'TeV     ', 'Data selection unit')
# hdu1.header.set('DSVAL3', '0.05:100', 'Data selection value')
# hdu1.header.set('NDSKEYS', '3       ', 'Number of data selections')
# hdu1.header

## Good Time Intervals (GTI)

### Header
<pre>
TSTART  =                      /start time same unit and system used in
                                the rate table
TSTOP   =                      /stop time same unit and system used in
                                the rate table
TIMEZERO=                      /zero time same unit and system used
                                in the rate table
TTYPE1  = 'START   '           / start of good time interval
TTYPE2  = 'STOP    '           / end of good time interval
EXTNAME = 'GTI     '           / name: Good Time Intervals
</pre>

### To-Do
1. Check that this is where the time cuts come from
2. exposure ?

### Parse time cut from cut config text.

In [ ]:
# Parse time cut string 
def parseTimeCut(tCutStr):
    cut_arr = []
    for cc in tCutStr.split(','):
        start = float(cc.split('/')[0])
        end   = float(cc.split('/')[1])
        cut_arr.append((start,end))
    return cut_arr

# Find time cut string
def getTimeCut(config_str_ori):
    config_str = str(config_str_ori)
    for i in config_str.splitlines():
        # Skip comment lines
        if( (len(i) ==0) or (i.strip()[0] == '#')):
            continue
        # I
        elif(i.find('ES_CutTimes') >= 0 ):
            key,cut_str = i.split(' ')
            if(len(cut_str) ==0):
                return parseTimeCut('0/0')
            return parseTimeCut(cut_str)
        
# Check if two cuts can be merged
def isMergable(cut1,cut2):
    s1,e1 = cut1
    s2,e2 = cut2
    if(s1 > s2):
        s1,e1 = cut2
        s2,e2 = cut1
    return (s2 <= e1)

# Merge two cuts
def mergeTwoTimeCut(cut1,cut2):
    s1,e1 = cut1
    s2,e2 = cut2
    if(s1 > s2):
        s1,e1 = cut2
        s2,e2 = cut1
    return (s1,max(e1,e2))


# Merge time cuts if there are overlaps
def mergeTimeCut(cuts):
    cut_sorted = sorted(cuts)
    merged_cut = []
    while(len(cut_sorted) > 0):
        test = cut_sorted[0]
        cut_sorted = cut_sorted[1:]
        the_rest = []
        for cc in cut_sorted:
            if(isMergable(test,cc)):
                test = mergeTwoTimeCut(test,cc)
            else:
                the_rest.append(cc)
        
        cut_sorted = the_rest
        merged_cut.append(test)
    return  list(filter(lambda x: x[0] != x[1],merged_cut))

# Get Good Time array from cuts
def getGTArray(startTime_s,endTime_s,cuts):
    if(len(cuts) ==0):
        return np.array([startTime_s]),np.array([endTime_s])
    goodTimeStart =  [startTime_s] + [ cc[1]+startTime_s for cc in cuts]
    goodTimeStop = [ cc[0]+startTime_s for cc in cuts] + [endTime_s]

    if(goodTimeStop[-1] >endTime_s):
        goodTimeStop[-1] = endTime_s
    if(goodTimeStart[-1] > goodTimeStop[-1]):
        return goodTimeStart[:-1],goodTimeStop[:-1]
    return np.array(goodTimeStart),np.array(goodTimeStop)



In [ ]:

runHeader = vegasFileIO.loadTheRunHeader()
cuts = vegasFileIO.loadTheCutsInfo()

startTime = runHeader.getStartTime()
endTime = runHeader.getEndTime()

startTime_s = float(startTime.getDayNS()) / 1e9
endTime_s = float(endTime.getDayNS()) / 1e9

# Get Time Cuts config string
for k in cuts:
    tmp =k.fCutsFileText
    tc = getTimeCut(k.fCutsFileText)
    
print('Time Cuts:',mergeTimeCut(tc))
goodTimeStart,goodTimeStop = getGTArray(startTime_s,endTime_s,mergeTimeCut(tc))
print('Start Time array',goodTimeStart)
print('Stop Time array',goodTimeStop)


In [ ]:
# do we record the unit in this table as well? (it is in seconds)

hdu2 = fits.BinTableHDU.from_columns([
    fits.Column(name='START', format='1D', array=goodTimeStart), 
    fits.Column(name='STOP', format='1D', array=goodTimeStop)
])
hdu2.name = "GTI"
hdu2.header.set('TSTART',startTime_s,'start time same unit and system used in the rate table')
hdu2.header.set('TSTOP',endTime_s,'stop time same unit and system used in the rate table')
hdu2.header.set('TIMEZERO',startTime_s,'zero time same unit and system used in the rate table') # is this correct ? 
hdu2.header.set('TTYPE1','START   ' ,' start of good time interval')
hdu2.header.set('TTYPE2','STOP    ' ,' start of good time interval')
hdu2.header.set('EXTNAME','GTI     ' ,' name: Good Time Intervals')

## Effective Area

Question - what do we need to do about sim vs real spectral index?

### First we need to load the EA file

In [ ]:
effectiveAreaIO = VARootIO(eaFile, True)
effectiveAreaIO.loadTheRootFile()

### Then we need to load the Effective Area

In [ ]:
effectiveAreaManager = VAEffectiveAreaManager()
effectiveAreaManager.loadEffectiveAreas(effectiveAreaIO)
effectiveAreaManager.setUseReconstructedEnergy(False)

### Calculate Average Noise of run

#### How does VEGAS decide what window size for noise calculation/look up ?

In [ ]:
avNoise = 0
nTels = 0
for telID in decodeConfigMask(runHeader.fRunInfo.fConfigMask):
    avNoise += qStatsData.getCameraAverageTraceVarTimeIndpt(telID-1, windowSizeForNoise, pixelData, arrayInfo)
    nTels += 1

avNoise /= nTels

### Set the EA parameters

In [ ]:
effectiveAreaParameters = VAEASimpleParameterData()
effectiveAreaParameters.fAzimuth = 0#(avAz)
effectiveAreaParameters.fZenith = 20#(90. - avAlt)
effectiveAreaParameters.fNoise = 5.3#avNoise # this still needs to be worked out
effectiveAreaParameters.fOffset = 0.5 # itterate over this in steps? (0.5deg as for sims?)

# convert to vector of parameters since this is required for a number of steps
effectiveAreaParameters = effectiveAreaManager.getVectorParamsFromSimpleParameterData(effectiveAreaParameters)


### Get the EA

In [ ]:
effectiveArea = effectiveAreaManager.getEffectiveAreaCurve(effectiveAreaParameters)

### Lets Check by Plotting

In [ ]:
x, y, ye = [], [], []
for i in range(effectiveArea.GetN()):
    tmpX, tmpY = ROOT.Double(0), ROOT.Double(0)
    effectiveArea.GetPoint(i, tmpX, tmpY)
    ye.append(effectiveArea.GetErrorY(i))
    effectiveArea.GetErrorX
    x.append(tmpX)
    y.append(tmpY)
        
x = np.array(x)
y = np.array(y)
ye = np.array(ye)

In [ ]:
# this can be removed from the later code but is good for checking
plt.errorbar(x, y, yerr = ye, ls = "", marker = "_")
plt.semilogy()

In [ ]:
c_float(0.5)

In [ ]:
energyLow = np.power(10, x - (x[1] - x[0])/2.)
energyHigh = np.power(10, x + (x[1] - x[0])/2.)
thetaLow = [0.0, 1.0]
thetaHigh = [1.0, 2.0]
# ea = np.vstack((y, y))
ea = [y,y]
minEnergy , maxEnergy = c_float(), c_float()
effectiveAreaManager.getSafeEnergyRange(effectiveAreaParameters, 0.5, minEnergy, maxEnergy)

In [ ]:
x = np.array([(energyLow, energyHigh, thetaLow, thetaHigh, ea)], 
             dtype=[('ENERG_LO', '>f4', np.shape(energyLow)), 
                    ('ENERG_HI', '>f4', np.shape(energyHigh)), 
                    ('THETA_LO', '>f4', np.shape(thetaLow)), 
                    ('THETA_HI', '>f4', np.shape(thetaHigh)), 
                    ('EFFAREA', '>f4', np.shape(ea))])

In [ ]:
hdu3 = fits.BinTableHDU(data=x)
hdu3.name = "EFFECTIVE AREA"

hdu3.header.set('TUNIT1 ', 'TeV', "")
hdu3.header.set('TUNIT2 ', 'TeV', "")
hdu3.header.set('TUNIT3 ', 'deg', "")
hdu3.header.set('TUNIT4 ', 'deg', "")
hdu3.header.set('TUNIT5 ', 'm^2', "")

hdu3.header.set('HDUCLASS', 'GADF',
                'FITS file following the GADF data format.')
hdu3.header.set('HDUCLAS1', 'RESPONSE', 'HDU class')
hdu3.header.set('HDUCLAS2', 'EFF_AREA', 'HDU class')
hdu3.header.set('HDUCLAS3', 'POINT-LIKE', 'HDU class')
hdu3.header.set('HDUCLAS4', 'AEFF_2D', 'HDU class')
hdu3.header.set('LO_THRES', minEnergy.value/1000.,
                'Low energy threshold of validity [TeV]')
hdu3.header.set('HI_THRES', maxEnergy.value/1000.,
                'High energy threshold of validity [TeV]')
hdu3.header.set('RAD_MAX ', 0.1, 'Direction cut applied [deg]')

hdu3.columns

In [ ]:
hdu3.header

## Migration Matrix
We want to get pfEnergy_Rec_VS_MC_2D for the correct parameters.

This could be done with a getter within VEGAS - lets try that first!

In [ ]:
#effectiveAreaManager.getEnergyBias2D(effectiveAreaParameters)
a, e = hist2array(effectiveAreaManager.getEnergyBias2D(effectiveAreaParameters), return_edges=True)

In [ ]:
eLow = np.power(10, [e[0][:-1]])[0]
eHigh = np.power(10, [e[0][1:]])[0]

bLow = np.power(10, [e[1][:-1]])[0]
bHigh = np.power(10, [e[1][1:]])[0]

ac = []
for aa in a:
    if np.sum(aa) > 0:
        ab = aa / np.sum(aa*(bHigh - bLow))
    else:
        ab = aa
    try:
        ac = np.vstack((ac, ab))
    except:
        ac = ab
        
ac = ac.transpose()

In [ ]:
# again good for checking - will delete later
plt.imshow(np.log10(ac))
plt.colorbar()

In [ ]:
x = np.array([(eLow, eHigh, bLow, bHigh, [0, 1.0], [1.0, 2.0], [ac, ac])], 
             dtype=[('ETRUE_LO', '>f4', (len(eLow),)), 
                    ('ETRUE_HI', '>f4', (len(eHigh),)), 
                    ('MIGRA_LO', '>f4', (len(bLow),)), 
                    ('MIGRA_HI', '>f4', (len(bLow),)), 
                    ('THETA_LO', '>f4', (2,)), 
                    ('THETA_HI', '>f4', (2,)), 
                    ('MATRIX', '>f4', (2, np.shape(ac)[0], np.shape(ac)[1]))])

hdu4 = fits.BinTableHDU(data=x)
hdu4.name = "ENERGY DISPERSION"

hdu4.header.set('TUNIT1 ', 'TeV', "")
hdu4.header.set('TUNIT2 ', 'TeV', "")
hdu4.header.set('TUNIT5 ', 'deg', "")
hdu4.header.set('TUNIT6 ', 'deg', "")
# hdu3.header.set('TUNIT5 ', 'm^2', "")

hdu4.header.set('HDUCLASS', 'GADF',
                'FITS file following the GADF data format.')
hdu4.header.set('HDUCLAS1', 'RESPONSE', 'HDU class')
hdu4.header.set('HDUCLAS2', 'EDISP', 'HDU class')
hdu4.header.set('HDUCLAS3', 'POINT-LIKE', 'HDU class')
hdu4.header.set('HDUCLAS4', 'EDISP_2D', 'HDU class')
hdu4.header.set('LO_THRES', minEnergy.value/1000.,
                'Low energy threshold of validity [TeV]')
hdu4.header.set('HI_THRES', maxEnergy.value/1000.,
                'High energy threshold of validity [TeV]')
hdu4.header.set('RAD_MAX ', 0.1, 'Direction cut applied [deg]')

hdu4.columns

In [ ]:
hdu4.header

## Write FITS file

In [ ]:
hdulist = fits.HDUList([hdu0, hdu1, hdu2, hdu3, hdu4])
hdulist.writeto(outfile, overwrite=True)